<a href="https://colab.research.google.com/github/LunaMing/MagentaTrain/blob/master/Magenta_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **先开！GPU！**
代码执行程序->更改运行时类型
在里面选GPU

# 安装magenta

In [0]:
#@test {"output": "ignore"}
%tensorflow_version 1.x

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

# 生成一个旋律（预训练模型）


记得导入attention_rnn.mag，就直接拖进来。

In [0]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file=/content/attention_rnn.mag \
--output_dir=/content/melody_rnn/generated \
--num_outputs=10 \
--num_steps=128 \
--primer_melody="[60]"

以“小星星”为旋律引子作曲；
就是添加了：60, -2, 60, -2, 67, -2, 67, -2

In [0]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file=/content/attention_rnn.mag \
--output_dir=/content/melody_rnn/generated \
--num_outputs=10 \
--num_steps=128 \
--primer_melody="[60, -2, 60, -2, 67, -2, 67, -2]"

# 构建数据集

新建一个mid_input文件夹，将要训练的mid文件上传

In [0]:
!convert_dir_to_note_sequences \
  --input_dir=/content/mid_input \
  --output_file=/content/notesequences.tfrecord \
  --recursive

# 创建SequenceExamples

In [0]:
!melody_rnn_create_dataset \
--config='attention_rnn' \
--input=/content/notesequences.tfrecord \
--output_dir=/content/melody_rnn/sequence_examples \
--eval_ratio=0.10

# 训练和验证模型

In [0]:
!melody_rnn_train \
--config=attention_rnn \
--run_dir=/content/melody_rnn/logdir/run1 \
--sequence_example_file=/content/melody_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir /content/melody_rnn/logdir